In [1]:
# Práctico Integrador – Predicción de Diabetes con Machine Learning
# Importar librerías

# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

# %%
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# %%
# Cargar dataset
ruta = '/content/drive/MyDrive/datasets/diabetes.csv'  # Ajustar según tu estructura de carpetas
df = pd.read_csv(ruta)

# %%
# Exploración inicial
print("Primeras filas del dataset:")
print(df.head())

print("\nInformación del dataset:")
print(df.info())

print("\nEstadísticas descriptivas:")
print(df.describe())

# %%
# Verificar valores faltantes
print("\nValores faltantes por columna:")
print(df.isnull().sum())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Primeras filas del dataset:
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column     

In [2]:
# ## 2. Preprocesamiento

# %%
# Reemplazar ceros por valores nulos en columnas donde no deberían existir
columnas_con_ceros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[columnas_con_ceros] = df[columnas_con_ceros].replace(0, np.nan)

# %%
# Reemplazar valores nulos por la mediana
for col in columnas_con_ceros:
    mediana = df[col].median()
    df[col].fillna(mediana, inplace=True)

# %%
# Verificar que no quedan valores nulos
print("Valores nulos después del tratamiento:")
print(df.isnull().sum())

# %%
# Separar características y variable objetivo
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# %%
# Normalizar los datos
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

# %%
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y, test_size=0.2, random_state=42
)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")

Valores nulos después del tratamiento:
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64
Tamaño del conjunto de entrenamiento: (614, 8)
Tamaño del conjunto de prueba: (154, 8)


/tmp/ipython-input-2-1005164641.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mediana, inplace=True)


In [3]:
# ## 3. Entrenamiento de modelos

# %%
# Inicializar modelos
modelos = {
    'Regresión Logística': LogisticRegression(max_iter=1000, random_state=42),
    'Árbol de Decisión': DecisionTreeClassifier(random_state=42),
    'Bosque Aleatorio': RandomForestClassifier(random_state=42)
}

# %%
# Entrenar y evaluar cada modelo
resultados = {}

for nombre, modelo in modelos.items():
    # Entrenar
    modelo.fit(X_train, y_train)

    # Predecir
    y_pred = modelo.predict(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    reporte = classification_report(y_test, y_pred)

    # Guardar resultados
    resultados[nombre] = {
        'modelo': modelo,
        'accuracy': accuracy,
        'confusion_matrix': cm,
        'classification_report': reporte
    }

    print(f"\n--- {nombre} ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Matriz de confusión:\n{cm}")
    print(f"Reporte de clasificación:\n{reporte}")


--- Regresión Logística ---
Accuracy: 0.7532
Matriz de confusión:
[[82 17]
 [21 34]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.80      0.83      0.81        99
           1       0.67      0.62      0.64        55

    accuracy                           0.75       154
   macro avg       0.73      0.72      0.73       154
weighted avg       0.75      0.75      0.75       154


--- Árbol de Decisión ---
Accuracy: 0.7143
Matriz de confusión:
[[76 23]
 [21 34]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.78      0.77      0.78        99
           1       0.60      0.62      0.61        55

    accuracy                           0.71       154
   macro avg       0.69      0.69      0.69       154
weighted avg       0.72      0.71      0.72       154


--- Bosque Aleatorio ---
Accuracy: 0.7403
Matriz de confusión:
[[78 21]
 [19 36]]
Reporte de clasificación:
              p

In [4]:
# ## 4. Evaluación y comparación

# %%
# Comparar accuracy de los modelos
print("\nComparación de modelos:")
for nombre, res in resultados.items():
    print(f"{nombre}: Accuracy = {res['accuracy']:.4f}")

# %%
# Identificar el mejor modelo
mejor_modelo_nombre = max(resultados, key=lambda x: resultados[x]['accuracy'])
mejor_modelo = resultados[mejor_modelo_nombre]['modelo']

print(f"\nEl mejor modelo es: {mejor_modelo_nombre} con accuracy = {resultados[mejor_modelo_nombre]['accuracy']:.4f}")

# ## 5. Exportación del modelo

# %%
# Guardar el mejor modelo
joblib.dump(mejor_modelo, 'mejor_modelo_diabetes.pkl')

# Guardar el escalador (importante para nuevos datos)
joblib.dump(scaler, 'escalador_diabetes.pkl')

# Descargar los archivos
from google.colab import files
files.download('mejor_modelo_diabetes.pkl')
files.download('escalador_diabetes.pkl')

print("Modelo y escalador exportados exitosamente")


Comparación de modelos:
Regresión Logística: Accuracy = 0.7532
Árbol de Decisión: Accuracy = 0.7143
Bosque Aleatorio: Accuracy = 0.7403

El mejor modelo es: Regresión Logística con accuracy = 0.7532


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Modelo y escalador exportados exitosamente
